# Curate sets of TARGET activity and selectivity data from ChEMBL

In [1]:
import importlib as imp
import pdb

import os
import sys
#sys.path.append(f"{os.environ['HOME']}/git/data_science/code")
import numpy as np
import pandas as pd

import curate_parp_data as cpd  # .py
import public_data_curation as pdc  # .py

#from atomsci.ddm.utils.curate_data import freq_table, aggregate_assay_data
import atomsci.ddm.utils.curate_data as cd
from atomsci.ddm.utils.rdkit_easy import setup_notebook
import atomsci.ddm.utils.data_curation_functions as dcf
import  atomsci.ddm.utils.struct_utils as su #import base_smiles_from_smiles, get_rdkit_smiles

from rdkit import Chem

import logging
logging.basicConfig(format='%(asctime)-15s %(message)s')
logger = logging.getLogger('ATOM')
logger.setLevel(logging.INFO)

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
matplotlib.rc('xtick', labelsize=12)
matplotlib.rc('ytick', labelsize=12)
matplotlib.rc('axes', labelsize=12)
%matplotlib inline
print("Imports Done.")

Imports Done.


In [2]:
setup_notebook()

Loading BokehJS ...

# Step 1: Collect Data

There are two methods to collect ChEMBL data:

1. Manually via web interface 
2. Via Python-API for ChEMBL (using SQL DB) - You need to install a ChEMBL 30 SQLite database on your system. 

## Option 1: Manual retrieval

ChEMBL datasets are expected to be downloaded from the ChEMBL website, https://www.ebi.ac.uk/chembl/ . 

The process for bioactivity data is: search for a target using its gene symbol; identify the correct Homo sapiens version of the target and click on its ChEMBL ID; click on the activity type you want (e.g., IC50) in the activity pie chart; then click on the CSV button above the result table to download the activity data. The downloaded CSV files are actually semicolon-separated. 

After uncompressing the downloaded file, rename it to indicate the target, activity type and species, with a ".txt" extension. Move the file to (root)/public_dsets/ChEMBL/raw, where (root) is /usr/local/data on TTB and /usr/workspace/atom on LC. 

For systems other than LC, you can use any location you want to. Feel free to create a directory '/Data' and use it as root. 

## Option 2: SQL Python ChEMBL API 

In [3]:
# # Create directory structure -> /Data
#                                      /chembl
#                                             /raw | /curated
dset_dir = f"Data/"
os.makedirs(dset_dir, exist_ok=True)
chembl_dir = f"{dset_dir}/chembl"
os.makedirs(chembl_dir, exist_ok=True)
raw_dir = f"{chembl_dir}/raw"
os.makedirs(raw_dir, exist_ok=True)
curated_dir = f"{chembl_dir}/curated"
os.makedirs(curated_dir, exist_ok=True)

In [4]:
log_var_map = {
    'IC50': 'pIC50',
    'AC50': 'pIC50',
    'Ki': 'pKi',
    'Solubility': 'logSolubility',
    'CL': 'logCL'
}
 # This is already saved at LC, you can download your own data and rename in similar format
chembl_dsets = dict(
    CYP2C9 = dict(AC50='CHEMBL25-CYP2C9_human_AC50_26Nov2019', IC50='CHEMBL25-CYP2C9_human_IC50_26Nov2019'),
    CYP2D6 = dict(AC50='CHEMBL25-CYP2D6_human_AC50_26Nov2019', IC50='CHEMBL25-CYP2D6_human_IC50_26Nov2019'),
    CYP3A4 = dict(AC50='CHEMBL25-CYP3A4_human_AC50_26Nov2019', IC50='CHEMBL25-CYP3A4_human_IC50_26Nov2019'),
    AURKA = dict(IC50="CHEMBL26-AURKA_human_IC50"),
    AURKB = dict(IC50="CHEMBL26-AURKB_human_IC50"),
    BSEP = dict(IC50="CHEMBL26-BSEP_human_IC50"),
    hERG = dict(IC50="CHEMBL26-hERG_human_IC50"),
    JAK1 = dict(IC50="CHEMBL25-JAK1_IC50_human_26Nov2019"),
    JAK2 = dict(IC50="CHEMBL25-JAK2_IC50_human_26Nov2019"),
    JAK3 = dict(IC50="CHEMBL25-JAK3_IC50_human_26Nov2019"),
    PI3Kg = dict(IC50="CHEMBL25-pI3K_p110gamma_human_IC50_26Nov2019"),
    KCNA5 = dict(IC50="CHEMBL26-KCNA5_human_IC50"),
    SCN5A = dict(IC50="CHEMBL26-SCN5A_human_IC50"),
    KCNQ1_KCNE1 = dict(IC50="CHEMBL26-KCNQ1_KCNE1_human_IC50"),
    CACNA1C = dict(IC50="CHEMBL26-CACNA1C_human_IC50"),
    CHRM1 = dict(IC50="CHEMBL26-CHRM1_human_IC50", Ki="CHEMBL26-CHRM1_human_Ki"),
    CHRM2 = dict(IC50="CHEMBL26-CHRM2_human_IC50", Ki="CHEMBL26-CHRM2_human_Ki"),
    CHRM3 = dict(IC50="CHEMBL26-CHRM3_human_IC50", Ki="CHEMBL26-CHRM3_human_Ki"),
    HRH1 = dict(IC50="CHEMBL26-HRH1_human_IC50", Ki="CHEMBL26-HRH1_human_Ki"),
    Solubility="CHEMBL25-Solubility_pH7_4_AstraZenica_26Nov2019",
    hepCL_human="CHEMBL25-hepatocyte_clearance_AZ_26Nov2019",
    hepCL_rat="CHEMBL25-hepatocyte_clearance_rat_AZ_26Nov2019",
    micCL_human="CHEMBL25-microsomal_clearance_human_AZprotocl_26Nov2019"
)
# Phospholipidosis dataset is not included above because the 'Standard Value' column is empty; it's a classification
# dataset where the classes are in the Comment column. Need to treat it specially.
chembl_dsets

{'CYP2C9': {'AC50': 'CHEMBL25-CYP2C9_human_AC50_26Nov2019',
  'IC50': 'CHEMBL25-CYP2C9_human_IC50_26Nov2019'},
 'CYP2D6': {'AC50': 'CHEMBL25-CYP2D6_human_AC50_26Nov2019',
  'IC50': 'CHEMBL25-CYP2D6_human_IC50_26Nov2019'},
 'CYP3A4': {'AC50': 'CHEMBL25-CYP3A4_human_AC50_26Nov2019',
  'IC50': 'CHEMBL25-CYP3A4_human_IC50_26Nov2019'},
 'AURKA': {'IC50': 'CHEMBL26-AURKA_human_IC50'},
 'AURKB': {'IC50': 'CHEMBL26-AURKB_human_IC50'},
 'BSEP': {'IC50': 'CHEMBL26-BSEP_human_IC50'},
 'hERG': {'IC50': 'CHEMBL26-hERG_human_IC50'},
 'JAK1': {'IC50': 'CHEMBL25-JAK1_IC50_human_26Nov2019'},
 'JAK2': {'IC50': 'CHEMBL25-JAK2_IC50_human_26Nov2019'},
 'JAK3': {'IC50': 'CHEMBL25-JAK3_IC50_human_26Nov2019'},
 'PI3Kg': {'IC50': 'CHEMBL25-pI3K_p110gamma_human_IC50_26Nov2019'},
 'KCNA5': {'IC50': 'CHEMBL26-KCNA5_human_IC50'},
 'SCN5A': {'IC50': 'CHEMBL26-SCN5A_human_IC50'},
 'KCNQ1_KCNE1': {'IC50': 'CHEMBL26-KCNQ1_KCNE1_human_IC50'},
 'CACNA1C': {'IC50': 'CHEMBL26-CACNA1C_human_IC50'},
 'CHRM1': {'IC50': 'CHEM

In [5]:
# ChEMBL target IDs for *human* Targets
# You need to manually find the CHEMBL ID for the specific target you want to explore.
# E.g. for CYP3A4, search for 'Cytochrome 450 3a4' in chenbl online database, then select chembl id 'CHEMBL340'.
chembl_target_id = dict(PARP1='CHEMBL3105', PARP2='CHEMBL5366', CYP3A4='CHEMBL340', CYP2C9='CHEMBL3397')
chembl_target_id

{'PARP1': 'CHEMBL3105',
 'PARP2': 'CHEMBL5366',
 'CYP3A4': 'CHEMBL340',
 'CYP2C9': 'CHEMBL3397'}

In [6]:
import chembl  # code/chembl.py which have functions for sql chembl db connector
import atomsci.ddm.utils.struct_utils as su

# This function was written to extract PARP data from ChEMBL, but this works for all targets. 
# target='PARP1' (default), you can call with any other target.
def get_raw_chembl_parp_activity_data(target='PARP1', activity_type='IC50', db_version='30', force_update=False):
    """
    Query ChEMBL for measurements of PARP1 or PARP2 activity of the given type.
    """
    raw_file = f"{raw_dir}/{target}_{activity_type}_chembl{db_version}.csv"
    if os.path.isfile(raw_file) and not force_update:
        print(f"Using existing raw {target} {activity_type} data from ChEMBL {db_version}")
        raw_df = pd.read_csv(raw_file)
        return raw_df
    # Note that the column names with chembl id was renamed while downloading, to distinguish multiple occurences.
    # Manual data curation may not have same column names. Before going to next step make sure you have same column names as this. I will write the column names in next step.
    print(f"Querying raw {target} {activity_type} data from ChEMBL {db_version}")
    with chembl.connect(version=db_version) as con:
        sql = ' '.join([
                     "SELECT DISTINCT act.activity_id, mol.chembl_id AS mol_chembl_id, cr.compound_name,",
                     "a.chembl_id AS assay_chembl_id, a.assay_type, a.assay_cell_type, a.assay_organism,",
                     "t.pref_name, a.description, act.standard_type,",
                     "act.standard_relation, act.standard_value, act.standard_units, cs.canonical_smiles,",
                     "doc.chembl_id AS doc_chembl_id, doc.title, doc.journal, doc.year, doc.volume, doc.issue,",
                     "doc.first_page, doc.last_page, doc.doi, doc.pubmed_id, doc.patent_id", 
                     "FROM assays a, activities act, compound_records cr, compound_structures cs,",
                     "target_dictionary t, molecule_dictionary mol, docs doc",
                     "WHERE a.tid = t.tid",
                     f"AND t.chembl_id = '{chembl_target_id[target]}'",
                     "AND act.assay_id = a.assay_id",
                     f"AND act.standard_type = '{activity_type}'",
                     "AND act.standard_value IS NOT NULL",
                     "AND act.standard_units IS NOT NULL",
                     "AND act.molregno = cr.molregno",
                     "AND act.molregno = mol.molregno",
                     "AND act.doc_id = doc.doc_id",
                     "AND cs.molregno = cr.molregno"])
        raw_df = pd.read_sql(sql, con=con)
        raw_df = raw_df.drop_duplicates(subset='activity_id')
  
#          # Add a reference column formatted the same way as in GoStar to facilitate checking for duplicates
# #         references = []
# #         for row in raw_df.itertuples():
# #             if row.volume is None:
# #                 volume = ''
# #             elif type(row.volume) == str:
# #                 volume = f"{float(row.volume):.0f}"
# #             else:
# #                 volume = f"{row.volume:.0f}"
# #             if row.issue is None:
# #                 issue = ''
# #             elif type(row.issue) == str:
# #                 issue = f"{float(row.issue):.0f}"
# #             else:
# #                 issue = f"{row.issue:.0f}"
# #             if row.journal is None:
# #                 journal = ''
# #             else:
# #                 journal = row.journal.replace('.', '')
# #             if not (row.patent_id is None):
# #                 references.append(row.patent_id)
# #             elif not (row.year is None):
# #                 references.append(f"{journal}, {row.year:.0f}, {volume} ({issue}), {row.first_page}-{row.last_page}")
# #             else:
# #                 references.append("")
# #         raw_df['reference'] = references
        
#         # Add a salt-stripped RDKit SMILES column. We only do that here so we can count unique compounds.
#         raw_df['base_rdkit_smiles'] = su.base_smiles_from_smiles(raw_df.canonical_smiles.values.tolist(), workers=16)
#         raw_df = raw_df[raw_df.base_rdkit_smiles != '']

#         # Map ChEMBL column names to corresponding DTC columns, so we can combine tables later, if needed
#         raw_df = raw_df.rename(columns={
#                         'mol_chembl_id': 'compound_id',
#                         'assay_chembl_id': 'assay_id',
#                         'assay_cell_type': 'cells_cellline_organ',
#                         'assay_organism': 'species',
#                         'pref_name': 'standard_name',
#                         'description': 'enzyme_cell_assay',
#                         'standard_type': 'activity_type',
#                         'standard_relation': 'activity_prefix',
#                         'standard_value': 'activity_value',
#                         'standard_units': 'activity_uom',
#                         'canonical_smiles': 'original_smiles',
#                         'doc_chembl_id': 'ref_id',
#                     })

#         raw_df.to_csv(raw_file, index=False)
#         print(f"Wrote {target} {activity_type}s to {raw_file}")

#         # Make a table of unique assays with record and compound counts
#         fr_df = cd.freq_table(raw_df, 'assay_id').rename(columns={'Count': 'assay_records'})
#         fr_df['assay_cmpds'] = [len(set(raw_df.loc[raw_df.assay_id == id, 'base_rdkit_smiles'].values)) 
#                                     for id in fr_df.assay_id.values]
#         assay_df = raw_df.drop_duplicates(subset='assay_id')
#         assay_df = assay_df.merge(fr_df, how='left', on='assay_id')
#         assay_file = f"{dset_dir}/{target}_{activity_type}_chembl{db_version}_assays.csv"
#         assay_df.to_csv(assay_file, index=False)
#         print(f"Wrote {target} assays to {assay_file}")

#         # Do the same for documents (references)
#         fr_df = cd.freq_table(raw_df, 'ref_id').rename(columns={'Count': 'ref_records'})
#         fr_df['ref_cmpds'] = [len(set(raw_df.loc[raw_df.ref_id == id, 'base_rdkit_smiles'].values)) 
#                                     for id in fr_df.ref_id.values]
#         ref_df = raw_df.drop_duplicates(subset='ref_id')
#         ref_df = ref_df.merge(fr_df, how='left', on='ref_id')
#         ref_file = f"{dset_dir}/{target}_{activity_type}_chembl{db_version}_refs.csv"
#         ref_df.to_csv(ref_file, index=False)
#         print(f"Wrote {target} references to {ref_file}")
#         print(f"{target}: {len(raw_df)} activities, {len(set(raw_df.base_rdkit_smiles.values))} compounds, {len(assay_df)} assays, {len(ref_df)} references")
#         
        raw_df.to_csv(raw_file, index=False)
        print(f"Wrote {target} {activity_type}s to {raw_file}")
        return raw_df

## Process raw activity data from ChEMBL

In [7]:
#imp.reload(cpd)
# Note: get_raw_chembl_parp_activity_data() ONLY WORKS FOR LC, AS THEY HAVE MYSQL AND CHEMBL INSTALLED !
get_raw_chembl_parp_activity_data('CYP2C9', 'IC50', force_update=True)
get_raw_chembl_parp_activity_data('CYP2C9', 'AC50', force_update=True)
get_raw_chembl_parp_activity_data('CYP2C9', 'EC50', force_update=True)
get_raw_chembl_parp_activity_data('CYP2C9', 'Ki', force_update=True)
get_raw_chembl_parp_activity_data('CYP2C9', 'Inhibition', force_update=True)
#get_raw_chembl_parp_activity_data('CYP2C9', 'CL', force_update=True)

Querying raw CYP2C9 IC50 data from ChEMBL 30
Wrote CYP2C9 IC50s to Data//chembl/raw/CYP2C9_IC50_chembl30.csv
Querying raw CYP2C9 AC50 data from ChEMBL 30
Wrote CYP2C9 AC50s to Data//chembl/raw/CYP2C9_AC50_chembl30.csv
Querying raw CYP2C9 EC50 data from ChEMBL 30
Wrote CYP2C9 EC50s to Data//chembl/raw/CYP2C9_EC50_chembl30.csv
Querying raw CYP2C9 Ki data from ChEMBL 30
Wrote CYP2C9 Kis to Data//chembl/raw/CYP2C9_Ki_chembl30.csv
Querying raw CYP2C9 Inhibition data from ChEMBL 30
Wrote CYP2C9 Inhibitions to Data//chembl/raw/CYP2C9_Inhibition_chembl30.csv


,activity_id,mol_chembl_id,compound_name,assay_chembl_id,assay_type,assay_cell_type,assay_organism,pref_name,description,standard_type,standard_relation,standard_value,standard_units,canonical_smiles,doc_chembl_id,title,journal,year,volume,issue,first_page,last_page,doi,pubmed_id,patent_id
0,111517,CHEMBL71472,Ethyl-sulfamic acid 5-tert-butyl-4-{4-hydroxy-...,CHEMBL660952,A,None,None,Cytochrome P450 2C9,Inhibition of Cytochrome P450 2C9 at 1 uM,Inhibition,=,0.0,%,CCNS(=O)(=O)Oc1cc(C(C)(C)C)c(SC2=C(O)OC(C)(CCc...,CHEMBL1131822,Nonpeptidic HIV protease inhibitors: 6-alkyl-5...,Bioorg. Med. Chem. Lett.,1999.0,9,15,2217,2222,10.1016/s0960-894x(99)00360-1,10465549.0,None
1,111518,CHEMBL71472,Ethyl-sulfamic acid 5-tert-butyl-4-{4-hydroxy-...,CHEMBL660951,A,None,None,Cytochrome P450 2C9,Inhibition of Cytochrome P450 2C9 at 10 uM,Inhibition,=,6.0,%,CCNS(=O)(=O)Oc1cc(C(C)(C)C)c(SC2=C(O)OC(C)(CCc...,CHEMBL1131822,Nonpeptidic HIV protease inhibitors: 6-alkyl-5...,Bioorg. Med. Chem. Lett.,1999.0,9,15,2217,2222,10.1016/s0960-894x(99)00360-1,10465549.0,None
2,111519,CHEMBL71472,Ethyl-sulfamic acid 5-tert-butyl-4-{4-hydroxy-...,CHEMBL660950,A,None,None,Cytochrome P450 2C9,Inhibition of Cytochrome P450 2C9 at 100 uM,Inhibition,=,46.0,%,CCNS(=O)(=O)Oc1cc(C(C)(C)C)c(SC2=C(O)OC(C)(CCc...,CHEMBL1131822,Nonpeptidic HIV protease inhibitors: 6-alkyl-5...,Bioorg. Med. Chem. Lett.,1999.0,9,15,2217,2222,10.1016/s0960-894x(99)00360-1,10465549.0,None
3,136755,CHEMBL285759,(S)-6-[2-(4-Amino-phenyl)-ethyl]-3-(2-tert-but...,CHEMBL660952,A,None,None,Cytochrome P450 2C9,Inhibition of Cytochrome P450 2C9 at 1 uM,Inhibition,=,2.0,%,Cc1cc(SC2=C(O)O[C@](CCc3ccc(N)cc3)(C(C)C)CC2=O...,CHEMBL1131822,Nonpeptidic HIV protease inhibitors: 6-alkyl-5...,Bioorg. Med. Chem. Lett.,1999.0,9,15,2217,2222,10.1016/s0960-894x(99)00360-1,10465549.0,None
5,136756,CHEMBL285759,(S)-6-[2-(4-Amino-phenyl)-ethyl]-3-(2-tert-but...,CHEMBL660948,A,None,None,Cytochrome P450 2C9,Inhibition of Cytochrome P450 2C9 at 10 uM,Inhibition,=,12.0,%,Cc1cc(SC2=C(O)O[C@](CCc3ccc(N)cc3)(C(C)C)CC2=O...,CHEMBL1131822,Nonpeptidic HIV protease inhibitors: 6-alkyl-5...,Bioorg. Med. Chem. Lett.,1999.0,9,15,2217,2222,10.1016/s0960-894x(99)00360-1,10465549.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4832,22976278,CHEMBL4798308,4-(Thiophen-2-yl)-1-(4-(4-(trifluoromethyl)phe...,CHEMBL4771893,A,None,Homo sapiens,Cytochrome P450 2C9,Inhibition of human CYP2C9 using fluorogenic s...,Inhibition,=,74.2,%,O=C(CCCc1cccs1)N1CCN(c2ccc(C(F)(F)F)cc2)CC1,CHEMBL4765370,"Design, synthesis, and biological evaluation o...",Eur J Med Chem,2020.0,202,None,112416,112416,10.1016/j.ejmech.2020.112416,32645646.0,None
4833,22983649,CHEMBL4799439,(S)-2-Methyl-4-(4-methylpiperazin-1-yl)-6-((2-...,CHEMBL4773470,A,None,Homo sapiens,Cytochrome P450 2C9,Inhibition of CYP2C9 in human liver microsomes...,Inhibition,=,1.0,%,Cc1nc(CN2CCC[C@H]2c2ncccc2C)cc(N2CCN(C)CC2)n1,CHEMBL4765416,"Design, synthesis, and evaluation of pyrrolidi...",Eur J Med Chem,2020.0,205,None,112537,112537,10.1016/j.ejmech.2020.112537,32768738.0,None
4834,22992013,CHEMBL4787096,(S)-3-(1-(5-fluoro-2-(1H-pyrazol-1-yl)phenyl)e...,CHEMBL4775386,A,None,Homo sapiens,Cytochrome P450 2C9,Inhibition of CYP2C9 (unknown origin) at 3 uM ...,Inhibition,<,50.0,%,Cc1n[nH]cc1-c1cnc(N)c(O[C@@H](C)c2cc(F)ccc2-n2...,CHEMBL4765473,PF-07059013: A Noncovalent Modulator of Hemogl...,J Med Chem,2021.0,64,1.0,326,342,10.1021/acs.jmedchem.0c01518,33356244.0,None
4835,23063435,CHEMBL4537788,BAY_784,CHEMBL4508920,B,None,Homo sapiens,Cytochrome P450 2C9,"CYP450, 2C9 Eurofins-Panlabs enzyme assay",Inhibition,None,32.0,%,O=C(NCc1ncc(C(F)(F)F)cc1Cl)c1ccc2c(c1)C1(CCS(=...,CHEMBL4507266,Data for DCP probe BAY-784,None,2021.0,None,None,None,None,10.6019/CHEMBL4507266,NaN,None


### At this point, we have downloaded ChEMBL data for different types of activities e.g. IC50, AC50, EC50, Ki, Inhibition percentage. 

#### Next, we need to check if columns of each downloaded CSV file is in same format and same names. 

In [7]:
target = 'CYP2C9'
rawdf_IC50 = pd.read_csv(f"{raw_dir}/{target}_IC50_chembl30.csv")
rawdf_IC50.columns

Index(['activity_id', 'mol_chembl_id', 'compound_name', 'assay_chembl_id',
       'assay_type', 'assay_cell_type', 'assay_organism', 'pref_name',
       'description', 'standard_type', 'standard_relation', 'standard_value',
       'standard_units', 'canonical_smiles', 'doc_chembl_id', 'title',
       'journal', 'year', 'volume', 'issue', 'first_page', 'last_page', 'doi',
       'pubmed_id', 'patent_id'],
      dtype='object')

In [8]:
rawdf_AC50 = pd.read_csv(f"{raw_dir}/{target}_AC50_chembl30.csv")
rawdf_AC50.columns

Index(['activity_id', 'mol_chembl_id', 'compound_name', 'assay_chembl_id',
       'assay_type', 'assay_cell_type', 'assay_organism', 'pref_name',
       'description', 'standard_type', 'standard_relation', 'standard_value',
       'standard_units', 'canonical_smiles', 'doc_chembl_id', 'title',
       'journal', 'year', 'volume', 'issue', 'first_page', 'last_page', 'doi',
       'pubmed_id', 'patent_id'],
      dtype='object')

In [9]:
rawdf_EC50 = pd.read_csv(f"{raw_dir}/{target}_EC50_chembl30.csv")
rawdf_EC50.columns

Index(['activity_id', 'mol_chembl_id', 'compound_name', 'assay_chembl_id',
       'assay_type', 'assay_cell_type', 'assay_organism', 'pref_name',
       'description', 'standard_type', 'standard_relation', 'standard_value',
       'standard_units', 'canonical_smiles', 'doc_chembl_id', 'title',
       'journal', 'year', 'volume', 'issue', 'first_page', 'last_page', 'doi',
       'pubmed_id', 'patent_id'],
      dtype='object')

In [10]:
rawdf_Inhibition = pd.read_csv(f"{raw_dir}/{target}_Inhibition_chembl30.csv")
rawdf_Inhibition.columns

Index(['activity_id', 'mol_chembl_id', 'compound_name', 'assay_chembl_id',
       'assay_type', 'assay_cell_type', 'assay_organism', 'pref_name',
       'description', 'standard_type', 'standard_relation', 'standard_value',
       'standard_units', 'canonical_smiles', 'doc_chembl_id', 'title',
       'journal', 'year', 'volume', 'issue', 'first_page', 'last_page', 'doi',
       'pubmed_id', 'patent_id'],
      dtype='object')

In [11]:
rawdf_Ki = pd.read_csv(f"{raw_dir}/{target}_Ki_chembl30.csv")
rawdf_Ki.columns

Index(['activity_id', 'mol_chembl_id', 'compound_name', 'assay_chembl_id',
       'assay_type', 'assay_cell_type', 'assay_organism', 'pref_name',
       'description', 'standard_type', 'standard_relation', 'standard_value',
       'standard_units', 'canonical_smiles', 'doc_chembl_id', 'title',
       'journal', 'year', 'volume', 'issue', 'first_page', 'last_page', 'doi',
       'pubmed_id', 'patent_id'],
      dtype='object')

In [13]:
#rawdf_CL = pd.read_csv(f"{raw_dir}/{target}_CL_chembl30.csv")
#rawdf_CL.columns

In [17]:
# check if the columns are same or not
if (rawdf_IC50.columns.all()) == (rawdf_AC50.columns.all()) and \
   (rawdf_AC50.columns.all()) == (rawdf_EC50.columns.all()) and \
   (rawdf_EC50.columns.all()) == (rawdf_Inhibition.columns.all()) and \
   (rawdf_Inhibition.columns.all()) == (rawdf_Ki.columns.all()):# and \
   #(rawdf_Ki.columns.all()) == (rawdf_CL.columns.all()):
    print("All activity dataframes have same columns")


All activity dataframes have same columns


#### Next, combine all activity data

In [18]:
activity_df_list = [rawdf_IC50, rawdf_AC50, rawdf_EC50, rawdf_Inhibition, rawdf_Ki] #, rawdf_CL]
raw_df = pd.concat(activity_df_list, ignore_index=True)
[raw_df.columns, raw_df.shape]

[Index(['activity_id', 'mol_chembl_id', 'compound_name', 'assay_chembl_id',
        'assay_type', 'assay_cell_type', 'assay_organism', 'pref_name',
        'description', 'standard_type', 'standard_relation', 'standard_value',
        'standard_units', 'canonical_smiles', 'doc_chembl_id', 'title',
        'journal', 'year', 'volume', 'issue', 'first_page', 'last_page', 'doi',
        'pubmed_id', 'patent_id'],
       dtype='object'),
 (14380, 25)]

In [19]:
raw_df.head(2)

,activity_id,mol_chembl_id,compound_name,assay_chembl_id,assay_type,assay_cell_type,assay_organism,pref_name,description,standard_type,standard_relation,standard_value,standard_units,canonical_smiles,doc_chembl_id,title,journal,year,volume,issue,first_page,last_page,doi,pubmed_id,patent_id
0,31876,CHEMBL152968,N-Biphenyl-3-yl-2-{4-[(R)-2-hydroxy-3-(2-methy...,CHEMBL660388,A,NaN,Cavia porcellus,Cytochrome P450 2C9,Inhibition of cytochrome P450 2C9 of isolated ...,IC50,=,24000.0,nM,Cc1nc2cc(OC[C@H](O)CN3CCN(CC(=O)Nc4cccc(-c5ccc...,CHEMBL1148425,New fatty acid oxidation inhibitors with incre...,Bioorg. Med. Chem. Lett.,2004.0,14.0,2,549.0,552.0,10.1016/j.bmcl.2003.09.093,14698201.0,NaN
1,45775,CHEMBL292759,4-Bromo-N-[4-methoxy-3-(4-methyl-piperazin-1-y...,CHEMBL666252,A,NaN,Homo sapiens,Cytochrome P450 2C9,Inhibitory potential of human cytochrome P450 ...,IC50,>,100000.0,nM,COc1ccc(NS(=O)(=O)c2ccc(Br)cc2)cc1N1CCN(C)CC1,CHEMBL1132410,5-Chloro-N-(4-methoxy-3-piperazin-1-yl- phenyl...,J. Med. Chem.,1999.0,42.0,2,202.0,205.0,10.1021/jm980532e,9925723.0,NaN


## Perform first stage of filtering and curation for ChEMBL 

In [20]:
# Make copies of the original activity types, prefixes, values and units, before we overwrite the columns with negative log
# transformed values
target='CYP2C9'
standard_type='IC50', 
db='chembl'
db_version='30'

raw_df['orig_standard_type'] = raw_df.standard_type.values
raw_df['orig_standard_relation'] = raw_df.standard_relation.values
raw_df['orig_standard_value'] = raw_df.standard_value.values
raw_df['orig_standard_units'] = raw_df.standard_units.values

### First, do some basic curation

In [21]:
# remane some columns to match general data curation functions available at AMPL for DTC and ExCape
raw_df = raw_df.rename(columns={
                        'mol_chembl_id': 'compound_id',
                        'assay_chembl_id': 'assay_id',
                        'pref_name': 'standard_name',
                        'doc_chembl_id': 'ref_id',
                        'canonical_smiles': 'original_smiles',
                    })
raw_df.columns

Index(['activity_id', 'compound_id', 'compound_name', 'assay_id', 'assay_type',
       'assay_cell_type', 'assay_organism', 'standard_name', 'description',
       'standard_type', 'standard_relation', 'standard_value',
       'standard_units', 'original_smiles', 'ref_id', 'title', 'journal',
       'year', 'volume', 'issue', 'first_page', 'last_page', 'doi',
       'pubmed_id', 'patent_id', 'orig_standard_type',
       'orig_standard_relation', 'orig_standard_value', 'orig_standard_units'],
      dtype='object')

In [24]:
# Add standardized desalted RDKit SMILES strings
raw_df['base_rdkit_smiles'] = su.base_smiles_from_smiles(raw_df.original_smiles.values.tolist(), workers=16) # Note: if you do not want to strip salt, please use ddm.utils.structUtils.get_rdkit_smiles(s)
raw_df = raw_df[raw_df.base_rdkit_smiles != '']

# # Add pIC50 values and filter on them
raw_df['pIC50'] = 9.0 - np.log10(raw_df.standard_value.values)
raw_df = raw_df[raw_df.pIC50 >= 3.0]

# # Add censoring relations for pIC50 values
rels = raw_df['standard_relation'].values
log_rels = rels.copy()
log_rels[rels == '<'] = '>'
log_rels[rels == '>'] = '<'
raw_df['pIC50_relation'] = log_rels

# get molecular weight using SMILES
mwts = [Chem.Descriptors.ExactMolWt(Chem.MolFromSmiles(x)) for x in raw_df.original_smiles.values.tolist()]
raw_df['molecular_weight'] = mwts

[14:55:22] Can't kekulize mol.  Unkekulized atoms: 3 5 26 27 28 29
[14:55:23] Can't kekulize mol.  Unkekulized atoms: 0 3 6
/g/g92/sarkar7/miniconda3/envs/AMPL-1.4.2/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log10
  
/g/g92/sarkar7/miniconda3/envs/AMPL-1.4.2/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in log10
  


In [25]:
raw_df.molecular_weight

0        516.219512
1        439.056525
2        516.219512
3        557.246061
4        487.055789
            ...    
14375    608.095091
14376    784.127179
14377    332.198759
14378    607.192387
14379    314.224580
Name: molecular_weight, Length: 14231, dtype: float64

In [26]:
# First curation step: Filter dataset to remove rows with missing IDs, SMILES, values, etc.
# Filter out rows with no SMILES string or IC50 data
raw_df = raw_df[~raw_df['base_rdkit_smiles'].isna()]
raw_df = raw_df[~raw_df['standard_value'].isna()]
raw_df.shape

(14231, 33)

#### Standardize relation and units

In [27]:
# Convert activity values to their negative log counterparts (pIC50 and pKi), taking the activity units into account
# first, check unique values
for col in raw_df:
    if col not in ['standard_type', 'standard_relation', 'standard_units']:
        continue
    print(col, raw_df[col].unique())

standard_type ['IC50' 'AC50' 'EC50' 'Inhibition' 'Ki']
standard_relation ['=' '>' '<' '>=' '<=' nan]
standard_units ['nM' 'min' 'ug.mL-1' '%' 'uM']


In [28]:
raw_df[raw_df['standard_type']=='IC50'].standard_units.unique()

array(['nM', 'min', 'ug.mL-1'], dtype=object)

In [29]:
raw_df[raw_df['standard_type']=='AC50'].standard_units.unique()

array(['nM'], dtype=object)

In [31]:
raw_df[raw_df['standard_type']=='EC50'].standard_units.unique()

array(['nM'], dtype=object)

In [32]:
raw_df[raw_df['standard_type']=='Inhibition'].standard_units.unique()

array(['%', 'uM'], dtype=object)

In [33]:
raw_df[raw_df['standard_type']=='Ki'].standard_units.unique()

array(['nM'], dtype=object)

#### Next, convert units
#### Question: How to convert '%', 'min', 'ug.mL-1'? 

In [34]:
unit_freq_df = cd.freq_table(raw_df, 'standard_units')
unit_freq_df

,standard_units,Count
2,nM,12529
0,%,1694
1,min,3
3,uM,3
4,ug.mL-1,2


In [24]:
# # Special case: Standard unit is nM, but some values are expressed as ug/mL, so convert them to nM
# in_ug_ml = (raw_df['standard_units'] == 'ug.mL-1')
# if (sum(in_ug_ml) > 0):
#     mwts = raw_df.loc[in_ug_ml, 'molecular_weight'].values
#     values_in_nM = raw_df.loc[in_ug_ml, 'standard_value'].values * 1e6 / mwts
#     raw_df.loc[in_ug_ml, 'standard_value'] = values_in_nM
#     raw_df.loc[in_ug_ml, 'standard_units'] = 'nM'

In [25]:
# """
# Standardize the censoring operators to =, < or >, and remove any rows whose operators
# don't map to a standard one.
# """
# raw_df['standard_relation'].fillna('=', inplace=True)
# ops = raw_df['standard_relation'].values
# # Remove annoying quotes around operators for ChEMBL DB
# ops = [op.lstrip("'").rstrip("'") for op in ops]
# op_dict = {
#     ">": ">",
#     ">=": ">",
#     "<": "<",
#     "<=": "<",
#     "=": "="
# }
# ops = np.array([op_dict.get(op, "@") for op in ops])
# raw_df['standard_relation'] = ops
# raw_df = raw_df[raw_df['standard_relation'] != "@"]

In [40]:
# # Convert activity values to their negative log counterparts (pIC50 and pKi), taking the activity units into account
# raw_df.loc[raw_df.standard_units.isin(['nM', 'nmol/L']), 'standard_units'] = 9.0 - np.log10(
#                                               raw_df.loc[raw_df.standard_units.isin(['nM', 'nmol/L']) and raw_df.standard_type.isin(['IC50', 'AC50']), 'standard_value'].values)

# raw_df.loc[raw_df.standard_units == 'uM', 'standard_value'] = 6.0 - np.log10(
#                                               raw_df.loc[raw_df.standard_units == 'uM', 'standard_value'].values)


# in_ug_ml = (dset_df['Standard Units'] == 'ug.mL-1')
# if (sum(in_ug_ml) > 0) and (max_unit == 'nM'):
#     mwts = dset_df.loc[in_ug_ml, 'Molecular Weight'].values
#     values_in_nM = dset_df.loc[in_ug_ml, 'Standard Value'].values * 1e6 / mwts
#     dset_df.loc[in_ug_ml, 'Standard Value'] = values_in_nM
#     dset_df.loc[in_ug_ml, 'Standard Units'] = 'nM'

    
# in_ic_ac = (raw_df['standard_type'].isin(['IC50', 'AC50'])
            
#     mwts = dset_df.loc[in_ic_ac, 'standard_units'].values
#     values_in_nM = dset_df.loc[in_ug_ml, 'Standard Value'].values * 1e6 / mwts
            
#     raw_df.loc[in_ug_ml, 'standard_value'] = values_in_nM
#     raw_df.loc[in_ug_ml, 'standard_units'] = 'nM'
            
            
# if (max_type in ['IC50', 'AC50']) and (max_unit == 'nM'):
#         raw_df['pIC50'] = 9.0 - np.log10(raw_df['standard_value'].values)
#         log_ops[ops == '>'] = '<'
#         log_ops[ops == '<'] = '>'

In [35]:
raw_df.standard_units.unique().tolist()

['nM', 'min', 'ug.mL-1', '%', 'uM']

In [36]:
unit_freq_df = cd.freq_table(raw_df, 'standard_units')
max_unit = unit_freq_df['standard_units'].values[0]
print(unit_freq_df)
print(max_unit)

  standard_units  Count
2             nM  12529
0              %   1694
1            min      3
3             uM      3
4        ug.mL-1      2
nM


In [37]:
type_df = cd.freq_table(raw_df, 'standard_type')
max_type = type_df['standard_type'].values[0]
print(type_df)
print(max_type)

  standard_type  Count
0          AC50   7280
2          IC50   5061
3    Inhibition   1697
4            Ki    159
1          EC50     34
AC50


In [38]:
[max_unit, max_type]

['nM', 'AC50']

In [39]:
# # Add a column for the pIC50 or log-transformed value, and a column for the associated censoring relation.
# # For pXC50 values, this will be the opposite of the original censoring relation.

# ops = raw_df['standard_relation'].values
# log_ops = ops.copy()

# for max_type in type_df['standard_type'].to_list():
#     print(max_type)
#     if (max_type in ['IC50', 'AC50']) and (max_unit == 'nM'):
#         raw_df['pIC50'] = 9.0 - np.log10(raw_df['standard_value'].values)
#         log_ops[ops == '>'] = '<'
#         log_ops[ops == '<'] = '>'
# elif (max_type == 'Ki') and (max_unit == 'nM'):
#     dset_df['pKi'] = 9.0 - np.log10(dset_df['Standard Value'].values)
#     log_ops[ops == '>'] = '<'
#     log_ops[ops == '<'] = '>'
# elif (max_type == 'Solubility') and (max_unit == 'nM'):
#     dset_df['logSolubility'] = np.log10(dset_df['Standard Value'].values) - 9.0
# elif max_type == 'CL':
#     dset_df['logCL'] = np.log10(dset_df['Standard Value'].values)
# dset_df['LogVarRelation'] = log_ops


In [40]:
# standard_type = 'IC50'
# # Standardize the relational operators. For activity types that were already negative log values (pIC50 or pKi, 
# # indicated by null units), keep the same relationships; otherwise, invert them.
# neglog_df = raw_df[raw_df.standard_units.isna()].copy()
# neglog_df = dcf.standardize_relations(neglog_df, rel_col='standard_relation', output_rel_col='standard_relation')

# needs_inv_df = raw_df[~raw_df.standard_units.isna()].copy()
# needs_inv_df = dcf.standardize_relations(needs_inv_df, rel_col='standard_relation', output_rel_col='standard_relation', invert=True)
# needs_inv_df['standard_type'] = f"p{standard_type}"

# raw_df = pd.concat([neglog_df, needs_inv_df], ignore_index=True)
# raw_df

In [51]:
# Everything is pIC50 or pKi now, so remove the units
raw_df['standard_units'] = np.nan

# convert smiles to standardize form 



In [30]:
# # Add URLs for Google Scholar or Google Patents queries on the references to make it easier to run searches from Excel
# # versions of the output files (using the HYPERLINK function)
# ref_urls = []
# for ref in raw_df.reference.values:
#     query = urllib.parse.quote_plus(ref)
#     if ref.startswith('US') or ref.startswith('WO'):
#         ref_urls.append(f"https://patents.google.com?oq={query}")
#     else:
#         ref_urls.append(f"https://scholar.google.com/scholar?hl=en&as_sdt=0&q={query}&btnG=")
# raw_df['ref_search_url'] = ref_urls

# # Write excluded and included records to separate files
# excluded_file = f"{dset_dir}/{target}_p{activity_type}_excluded_{db}_{db_version}.csv"
# excluded_df.to_csv(excluded_file, index=False)
# print(f"Wrote excluded {db} data to {excluded_file}")

# filtered_file = f"{dset_dir}/{target}_p{activity_type}_filtered_{db}_{db_version}.csv"
# filtered_df.to_csv(filtered_file, index=False)
# print(f"Wrote filtered {db} data to {filtered_file}")

In [29]:
# imp.reload(cpd)
# chembl_filt_df = cpd.curate_parp_activity_data(chembl_df, target='CYP2C9', activity_type='IC50', db='chembl', db_version='30')

In [52]:
raw_df.columns # final raw df

Index(['activity_id', 'compound_id', 'compound_name', 'assay_id', 'assay_type',
       'assay_cell_type', 'assay_organism', 'standard_name', 'description',
       'standard_type', 'standard_relation', 'standard_value',
       'standard_units', 'canonical_smiles', 'ref_id', 'title', 'journal',
       'year', 'volume', 'issue', 'first_page', 'last_page', 'doi',
       'pubmed_id', 'patent_id', 'orig_standard_type',
       'orig_standard_relation', 'orig_standard_value', 'orig_standard_units'],
      dtype='object')

#### Next, remove outliers and bad duplicates using AMPL

In [42]:
#imp.reload(cpd)
filt_df = cd.remove_outlier_replicates(raw_df, response_col='standard_value', max_diff_from_median=0.5)
raw_df.shape, filt_df.shape

Removed 946 standard_value replicate measurements that were > 0.5 from median


((14231, 33), (13285, 33))

#### Next, aggregate assay data using AMPL

Note that, active threshold 6.0 is used here. This can be changed based on application and domain knowledge. For drug discovery, usually molecules with pIC50 value 6 or less are good drug candidates. 

In [44]:
agg_df = cd.aggregate_assay_data(filt_df, value_col='standard_value', active_thresh=6.0, id_col='compound_id',
                                   smiles_col='base_rdkit_smiles', relation_col='standard_relation')
agg_df.shape

[14:59:23] Can't kekulize mol.  Unkekulized atoms: 3 5 26 27 28 29
[14:59:28] Can't kekulize mol.  Unkekulized atoms: 0 3 6


(13083, 5)

In [45]:
agg_df.head()

,compound_id,base_rdkit_smiles,relation,standard_value,active
0,CHEMBL1591190,CN(C)c1ncc2ncc(=O)n(Cc3cccs3)c2n1,,39810.72,1
1,CHEMBL1599663,Cc1cc(C)n(-c2cc(N3CCN(C(=O)c4cccc(Br)c4)CC3)cc...,,5623.41,1
2,CHEMBL1382227,Cc1ccc(NC(=O)Cn2nc([N+](=O)[O-])c(Br)c2C)cc1C,,14125.38,1
3,CHEMBL4516287,C=CC(=O)Nc1cc(Nc2nccc(Nc3ccccc3S(=O)(=O)C(C)C)...,,90.00,1
4,CHEMBL1335690,COc1ccc(C2Nc3ccccc3C(=O)N2Cc2ccco2)cc1COc1ccc(...,,177.83,1


In [46]:
cur_file = f"{dset_dir}/{target}_chembl_pIC50_curated.csv"
filt_df.to_csv(cur_file, index=False)
print(f"Wrote {cur_file}")

Wrote Data//CYP2C9_chembl_pIC50_curated.csv


In [47]:
agg_file = f"{dset_dir}/{target}_chembl_pIC50_agg.csv"
agg_df.to_csv(agg_file, index=False)
print(f"Wrote {agg_file}")

Wrote Data//CYP2C9_chembl_pIC50_agg.csv


In [48]:
cd.freq_table(agg_df, 'relation')

,relation,Count
0,,10526
2,>,2235
1,<,322
